# Carga de la fuente

In [1]:
#Importación de la librería NLTK
import nltk
from nltk.corpus import PlaintextCorpusReader
nltk.download('cess_esp')
nltk.download('conll2002')
nltk.download('omw')
nltk.download('punkt')

#Definición de la locación
from google.colab import drive
drive.mount('/content/drive')

#Definición del origien y nombre del archivo
root='/content/drive/MyDrive/npl_lel/fuentes/fuente_casos'
file_name='fuente_2.txt'

#Definición del corpus
corpus=PlaintextCorpusReader(root, file_name)

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.
[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Detección de símbolos potenciales

In [2]:
#Descarga e instalacion de librerías y modelos
!pip install git+https://github.com/boudinfl/pke.git

!pip install spacy
!spacy download es_core_news_sm

import spacy
import pke
import es_core_news_sm

import string
from spacy.lang.es.stop_words import STOP_WORDS

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-zg9hy1e_
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-zg9hy1e_
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.2 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160627 sha256=3e563baf7e5065693ec2c9f5c8d84b4245a7f6c456c0369b3ca5d9c13a6d57cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-vbnpiq5y/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke
2023-08-12 23:46:20.965884: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appro

In [3]:
#Definición del modelo spacy
nlp=spacy.load('es_core_news_sm')

#Definición de stopwords
stopwords = nlp.Defaults.stop_words
puntuacion= string.punctuation
for p in puntuacion:
  stopwords.add(p)


#Definición del texto
fuente=nlp(corpus.raw())

#Definición del extractor
extractor = pke.unsupervised.TextRank()

#Carga del texto
extractor.load_document(input=fuente,language="es",stoplist=stopwords,spacy_model=nlp)

In [4]:
# Definición de Keywords candidatas
extractor.candidate_selection(pos=None)

In [5]:
# Construcción del grafo y ranking de los vértices
extractor.candidate_weighting()

#Definción de la cantidad de keyphrases como T, donde T equivale a un tercio de los vértices)
t=round(len(extractor.candidates)/3)
keyphrases= extractor.get_n_best(t)

#Defino Símbolos Potenciales
simbolos_potenciales=[]
for k in keyphrases:
  simbolos_potenciales.append(k[0])

In [6]:
len(simbolos_potenciales)

42

# Limpieza de símbolos potenciales

In [7]:
def get_pos(simbolo):#le doy el token  y me devuelve el pos
  for token in fuente:
    if str(simbolo) == token.text:
      return token.pos_

def get_dep(simbolo):#le doy el token  y me devuelve el dep
  for token in fuente:
    if str(simbolo) == token.text:
      return token.dep_

def is_child(parent,child):#devuelve un boolean de ser los tokens parent y child efectivamente padre e hijo
  for token in fuente.__iter__():
    if str(parent) == token.text:
      for t_child in token.children:
        if(str(child).lower()==str(t_child.text).lower()):
          return True
          break;
  return False

def multiple_objs(sp):#Devuelve si el simbolo tiene multiples tokens objeto
  nro_t_objetos=0 #nro de tokens de tipo objeto en el símbolo potencial
  for t in nlp(sp):
    if(get_dep(t)=="obj"):
      nro_t_objetos=nro_t_objetos+1
    if(nro_t_objetos>1):
      return True
  return False

#Quitar los símbolos que sean iguales en la lista de símbolos potenciales y dejar una sola instancia del símbolo.
def quitar_simbolos_repetidos():
  for sp_1 in simbolos_potenciales:
    i=0
    for sp_2 in simbolos_potenciales:
      if(sp_1==sp_2):
        i=i+1
        if(i>1):
          simbolos_potenciales.remove(sp_2)

#Quitar los símbolos vacíos en la lista de símbolos potenciales
def quitar_simbolos_vacios():
  for sp in simbolos_potenciales:
    if(sp==""):
      simbolos_potenciales.remove(sp)


In [8]:
#Fase 1: detectar múltiples símbolos en keyphrases --> si hay varios los separo
def separar_simbolos():                                                     #1. Si mi sp tiene múltiples objs en DEP entonces tengo dos símbolos.
  list_new_symbols=[]
  insertions = []
  for sp in simbolos_potenciales:
   if multiple_objs(sp):                                                    #Tengo más de un símbolo en el símbolo potencial
      nlp_sp = nlp(sp)
      for t in nlp_sp:                                                      #Para el token en el símbolo
        if get_dep(t)=="obj" or get_dep(t)=="iobj" or get_dep(t)=="dobj":   #Si mi token es un objeto. Version anterior: SIN  or get_dep(t)="iobj" or get_dep(t)="dobj":
          nuevo_simbolo=""                                                  #Entonces hay un nuevo símbolo.
          for i in nlp_sp:                                                  #Así que para todos los tokens que componen a mi símbolo
            if is_child(t,i) or i==t:                                       # Si alguno de ellos es hijo de mi token objeto o es mi mismo token objeto
              if nuevo_simbolo=="" :
                nuevo_simbolo=i.text                                        #Entonces es parte de mi nuevo símbolo
              else:
                nuevo_simbolo=nuevo_simbolo+str(" ")+i.text
          list_new_symbols.append(nuevo_simbolo)
      index=simbolos_potenciales.index(sp)
      simbolos_potenciales.pop(index)                                       #Una vez que terminé de analizar todos los tokens obj de mi símbolo orignal, quito mi símbolo orignal de los símbolos potenciales
      for i, new_symbol in enumerate(list_new_symbols):
        insertions.append((index + i, new_symbol))

  list_new_symbols.clear()

  for idx, new_symbol in insertions:
    simbolos_potenciales.insert(idx, new_symbol)

  insertions.clear()

  quitar_simbolos_repetidos()
  quitar_simbolos_vacios()

#Fase 2: Si el lemma del token en un símbolo es una stopword, entonces quito el token
def remove_token(sp, token):
    simbolo = str(sp).split()
    nuevos_tokens = [t for t in simbolo if t != token]
    simbolo_actualizado = ' '.join(nuevos_tokens)
    return simbolo_actualizado

def is_lemma_stopword(t):# Si el lemma del token en un símbolo es una stopword, entonces quito el token
  nlp = spacy.load('es_core_news_sm')
  doc = nlp(t)
  if doc[0].lemma_ in stopwords:
      return True
  return False

def quitar_simbolos_stopword():
  simbolos_a_quitar=[]
  for sp in simbolos_potenciales:
    nlp_sp = nlp(sp)
    for t in nlp_sp:                                                                            #Para el token en el símbolo
      if is_lemma_stopword(str(t)):
        simbolos_potenciales[simbolos_potenciales.index(sp)]=remove_token(str(sp),str(t))
        if str(sp)==str(t):                                                                             #Si token es todo el símbolo, entonces quito el símbolo
          sp=remove_token(str(sp),str(t))
          simbolos_potenciales.remove(sp)
        else:
          sp=remove_token(str(sp),str(t))
  quitar_simbolos_repetidos()
  quitar_simbolos_vacios()


In [9]:
separar_simbolos()
quitar_simbolos_stopword()

In [10]:
print(simbolos_potenciales)

['columna “ autoevaluación ”', 'columna “ evaluación ”', 'enviarles email informándoles', 'empleados asociados', 'supervisores', 'supervisores asociados', 'estándar smart', 'explicado', 'información', 'feedback', 'autoevaluación vencido', 'cargo asociado', 'objetivos asociados', 'cargo', 'periodo', 'empleados registrados', 'autoevaluación', 'información previa', 'fecha mínima', 'fecha máxima', 'competencias específicas', 'feedback vencido', 'supervisión vencido', 'empleado seleccionado', 'objetivos específicos', 'texto libre', 'tiempos establecidos', 'tarea identifica', 'única', 'configuración iniciales', 'link provisto', 'proceso integral', 'aspectos generales', 'columna feedback', 'actividades', 'supervisión', 'competencias genéricas', 'supervisor detectado', 'asociado', 'evaluación completo']


In [11]:
len(simbolos_potenciales)

40

# Iniciación y nombramiento  de los símbolos

In [12]:

#Definición de la clase símbolo
class Simbolo:
  nombre=""
  clase=""
  nocion=[]
  impacto=[]
  otros_nombres=[]
  especializaciones=[]
  es_referenciado=False
  hace_referencia=False

  def __init__(self, nombre):
    self.nombre = nombre


In [13]:
simbolos_confirmados= []
for simbolo in simbolos_potenciales:
  simbolos_confirmados.append(
      Simbolo(simbolo)
  )

del(simbolos_potenciales)

In [14]:
len(simbolos_confirmados)

40

# Clasificación de los símbolos

In [15]:
def symbol_has_root(nlp_sc):#Devuelve un bool especificando si el símbolo tiene un token ROOT
  for t in nlp_sc:
    if t.dep_=="ROOT":
      return True
  return False

def symbol_get_root(nlp_sc): #Devuelve el token ROOT del símbolo.
  for t in nlp_sc:
    if t.dep_=="ROOT":
      return t
  return None

def is_named_entity(sc): #Devuelve un bool especificando si el símbolo es una de las Named Entities en la fuente
  named_entities = [(entity.text) for entity in fuente.ents]
  for n_e in named_entities:
    if sc.nombre.lower()==n_e.lower():
      return True
  return False

In [16]:
def clasificar_token_pos(pos_tag):#Define cómo clasifico a un símbolo en base a sus POS tags:
  if pos_tag==None:
    return None
  else:
    if pos_tag.startswith("VERB"):
      return "VERBO"
    elif pos_tag.startswith("NOUN"):
      return "SUJETO"
    elif pos_tag in ["PRON", "DET"]:
      return "OBJETO"
    elif pos_tag.startswith("ADJ"):
      return "ESTADO"
    else:
      return None

def clasificar_token_dep(dep_tag):                                  #Define cómo clasifico a un símbolo en base a sus DEP tags:
  if dep_tag in ["ROOT", "advcl", "ccomp"]:
    return "VERBO"
  elif dep_tag in ["nsubj", "nsubjpass"]:
    return "SUJETO"
  elif dep_tag in ["obj","dobj", "iobj", "pobj", "attr"]:
    return "OBJETO"
  elif dep_tag in ["amod","advmod"]:
    return "ESTADO"
  else:
    return None

In [17]:
#Intenta determinar la clase del símbolo en base a la DEP tag asignado al token root en la fuente
#Si esta es nula, intenta determinar la clase del símbolo en base a la DEP tag asignado al token root en el contexto del símbolo
def definir_clase_por_dep(dep_tag,root):
  clase=clasificar_token_dep(dep_tag)
  if clase==None:
    clase=clasificar_token_dep(root.dep_)
  return clase

#Intenta determinar la clase del símbolo en base a la POS tag asignado al token root en la fuente
#Si esta es nula, intenta determinar la clase del símbolo en base a la POS tag asignado al token root en el contexto del símbolo
def definir_clase_por_pos(pos_tag,root):
  clase=clasificar_token_pos(pos_tag)
  if clase==None:
    clase=clasificar_token_pos(root.pos_)
  return clase

#Determina la clase del símbolo
def definir_clase(sc,nlp_sc):
  clase=None
  if is_named_entity(sc): #if sc es una Named entity, entonces mi símbolo es un SUJETO
    clase="SUJETO"
  if symbol_has_root(nlp_sc):
    root=symbol_get_root(nlp_sc)
    clase_dep=definir_clase_por_dep(get_dep(root),root)
    clase_pos=definir_clase_por_pos(get_pos(root),root)
    if clase_dep==clase_pos:
      clase=clase_dep# adicionar if clase=None en la llamada de la funcion
    elif clase_dep==None:
      clase=clase_pos
    elif clase_pos==None:
      clase=clase_dep
    elif clase_dep=="VERBO":
      if clase_pos=="SUJETO" or clase_pos=="OBJETO":
        clase="OBJETO"
      else:
        clase=clase_pos
    elif clase_dep=="OBJETO" or clase_dep=="SUJETO" or clase_dep=="ESTADO":
      clase=clase_dep
    if clase==None:
      print("SM",nlp_sc.text,"CLASE_DEP: ",clase_dep,"CLASE_POS: ",clase_pos)
  sc.clase=clase

In [18]:
#Llama a determinar la clase del símbolo.
#Si la clase del símbolo resulta ser nula, entonces quito el símbolo de la lista de símbolos confirmados
#
for sc in simbolos_confirmados:
  nlp_sc=nlp(sc.nombre)
  definir_clase(sc,nlp_sc)
  if sc.clase==None: # Una vez clasificado, quito el simbolo si no aplican
    simbolos_confirmados.remove(sc)

In [19]:
len(simbolos_confirmados)

40

# Noción e Impacto de los símbolos


In [20]:
def get_sent(sc_name):                          #Devuelve una lista de oraciones que incluyen al símbolo parámetro
  sentences=[]
  for sent in fuente.sents:
    if sc_name in sent.text.lower():
      sentences.append(sent.text)
  return sentences

In [21]:
def quitar_oraciones_repetidas(sentences):      #De encontrarlas, quitas las instancias repetidas de una frase
  for sent_1 in sentences:
    i=0
    for sent_2 in sentences:
      if(sent_1==sent_2):
        i=i+1
        if(i>1):
          sentences.remove(sent_2)
  return sentences

def describir_simbolo(simbolo):
  sentences=get_sent(simbolo.nombre)# Oraciones que contengan el nombre del símbolo
  nocion = []
  impacto = []

  for sentence in sentences:
    doc = nlp(sentence)
    nlp_sc=nlp(simbolo.nombre.lower())
    root=symbol_get_root(nlp_sc)
    for token in doc:
      if str(token.text.lower()) == str(root):
        if clasificar_token_dep(token.dep_) == "SUJETO":
          nocion.append(sentence)
        elif clasificar_token_dep(token.dep_) == "OBJETO":
          impacto.append(sentence)
        elif clasificar_token_dep(token.dep_) == "ESTADO":
          nocion.append(sentence)
        elif clasificar_token_dep(token.dep_) == "VERBO":
          impacto.append(sentence)
        else:
          impacto.append(sentence)

  nocion=quitar_oraciones_repetidas(nocion) #De encontrarlas, quitas las instancias repetidas de una frase
  impacto=quitar_oraciones_repetidas(impacto) #De encontrarlas, quitas las instancias repetidas de una frase
  simbolo.nocion=nocion
  simbolo.impacto=impacto

def describir_simbolos_confirmados():
  for sc in simbolos_confirmados:
    describir_simbolo(sc)
  for sc in simbolos_confirmados: #Quitar símbolos con nociones e impactos vacios
    if len(sc.nocion)==0:
      if len(sc.impacto)==0:
        simbolos_confirmados.remove(sc)

describir_simbolos_confirmados()


In [22]:
len(simbolos_confirmados)

39

# Referenciacion entre símbolos

In [23]:
#1. Subraya los casos de menciones en otros símbolos. #. Si hay un símbolo con 0 menciones. Quitarlo del LEL final.

def marcar_referencias(simbolo):  #Marca las referencias del simbolo en otros simbolos, en estos otros simbolos donde se hacen referencias marca el atributo hace_referencia como true.
#Devuelve un boolean true si se marcaron referencias al símbolo en otros símbolos, y false si no se encontraron referencias.
  es_referenciado=False
  nombre=simbolo.nombre
  for sc in simbolos_confirmados:
    if nombre!=sc.nombre: #mientras no se trate del mismo simbolo
      for noc in sc.nocion:
        if nombre in noc:
          sc.nocion[sc.nocion.index(noc)]= noc.replace(nombre, "\033[4m" + nombre + "\033[0m")
          es_referenciado=True #mi simbolo es referenciado por otro
          sc.hace_referencia=True #y el otro simbolo hace refrencias
      for imp in sc.impacto:
        if nombre in imp:
          sc.impacto[sc.impacto.index(imp)]=imp.replace(nombre, "\033[4m" + nombre + "\033[0m")
          es_referenciado=True
          sc.hace_referencia=True
  return es_referenciado

def quitar_simbolos_aislados():                            #Quita los símbolos que no hacen referencias o no son referenciados por otros símbolos.
  for sc in simbolos_confirmados:
    if sc.es_referenciado==False:
      simbolos_confirmados.remove(sc)
    else:
      if sc.hace_referencia==False:
        simbolos_confirmados.remove(sc)

def referenciacion_de_simbolos():                       #Define el proceso de referencias entre los símbolos
  for sc in simbolos_confirmados:                       #Marco todas las referencias entre los simbolos
    sc.es_referenciado=marcar_referencias(sc)
  quitar_simbolos_aislados()                            #Una vez marcadas las referencias quito los símbolos que no tienen relacion con el resto del LEL (es decir, estan aislados)

referenciacion_de_simbolos()


In [24]:
len(simbolos_confirmados)

31


# Simbolos sinonimos


In [25]:
#2. Revisar si hay algunos símbolos que sean sinónimos

def asimilar_sinonimos():                                                                         #Si dos simbolos tienen igual clase, nocion e impacto los considero sinónimos y los vuelvo un solo símbolo
  for sc_1 in simbolos_confirmados:
    sinonimos=[]
    for sc_2 in simbolos_confirmados:
      if sc_1.nombre!=sc_2.nombre:
        if sc_1.clase==sc_2.clase and sc_1.nocion==sc_2.nocion and sc_1.impacto==sc_2.impacto:    #Son sinonimos
          sinonimos.append(str(sc_2.nombre))
          simbolos_confirmados.remove(sc_2)                                                       #Quito uno de los símbolos sinónimos de la lista de simbolos.
    sc_1.otros_nombres= sinonimos                                                                 #SUmo los otros nombres del símbolo

asimilar_sinonimos()

In [26]:
len(simbolos_confirmados)

31

# Jerarquías entre simbolos

In [27]:
def es_especializacion(simbolo, especializacion):#Devuelve un boolean de si el parametro especializacion es o no una especializacion del parámetro símbolo# Si el nombre o algún nombre sinónimo de sc_1 es igual a la root de otros símbolos entonces sc_2 es una especialización ("hijo") de sc_1#Armo dos listas con los nombres de cada simbolo.
  simbolo_nombres=[]
  simbolo_nombres.append(simbolo.nombre)
  especializacion_nombres=[]
  especializacion_nombres.append(especializacion.nombre)
  for otro_nombre in simbolo.otros_nombres:
    simbolo_nombres.append(otro_nombre)
  for otro_nombre in especializacion.otros_nombres:
    especializacion_nombres.append(otro_nombre)    #Checkeo si alguno de los nombres de la especializacion tienen como root a alguno de los nombre del simbolos
  for esp in especializacion_nombres:
    if str(symbol_get_root(nlp(esp))) in simbolo_nombres: #Si si, entonces devuelvo TRUE: es una especializacion
      return True
  return False #Si no, entonces devuelvo FALSE: no es una especialización

#Si un símbolo es la root de otros símbolos--> están en una jerarquía taxonómica.
def definir_jerarquias():
  for sc_1 in simbolos_confirmados:
    esps=[]
    for sc_2 in simbolos_confirmados:
      if str(sc_1.nombre)!=str(sc_2.nombre):    #Estoy haciendo un loop anidado, asi que si es mismo simbolo entonces no lo evaluo
        # Si el nombre o algún nombre sinónimo de sc_1 es igual a la root de otros símbolos
        # entonces sc_2 es una especialización ("hijo") de sc_1
        if es_especializacion(sc_1,sc_2): #Si sc_2 es una especializacion de sc_1, estan en una jerarquia taxonómica, y sumo a sc_2 como especializcion de sc_1
          esps.append(sc_2)
    sc_1.especializaciones=esps

definir_jerarquias()



In [28]:
len(simbolos_confirmados)

31

# Configuración de los nombres


In [29]:
#Reglas de configuración de los nombres, en base a Proceso de Creación del LEL :
# Regla 1. Debe utilizarse la forma singular el nombre del símbolo, excepto en casos en que la versión singular y la plural no se usan con el mismo significado
# o si la forma singular no tiene sentido en el contexto de la aplicación.
#Regla 2: Hay que sacar los que hayan quedado mal configurados y poner los en infinitivo a los verbos.

#En ambos casos, es decir tanto para que una palabra quede en singular como en infinitivo, la palabra debería ser igual a su lemma.
#En el caso de los símbolos "ESTADO", esto no debería aplicar.

def es_lemma(nombre):
  if nombre.text==nombre.lemma_:
    return True
  return False

def configurar_nombres():
  for sc in simbolos_confirmados:
    if sc.clase!="ESTADO":
      nombre=nlp(sc.nombre)
      for token in nombre:                                        #Para cada token que compone al nombre checkeo si es igual a su lemma
        if es_lemma(token)==False:
          sc.nombre=sc.nombre.replace(token.text,token.lemma_)

configurar_nombres()



# Simbolos Homónimos

In [30]:
#Una vez reconfigurados los nombres de los símbolos, si dos símbolos quedaron con el mismo nombre,
#pero como ya pasada la instancia de chequeo de sinónimos sé que efectivamente no se trata de sinónimos porque no coinciden en clase o descripción
#Entonces considero a estos símbolos como símbolos homonimos.

# Impresión de LEL

In [42]:
from google.colab import files

def get_simbolo_string(simbolo):
    output = "\nNOMBRE: " + simbolo.nombre + "\n"
    if simbolo.otros_nombres:
        output += "Otros nombres:\n"
        for nom in simbolo.otros_nombres:
            output += "- " + nom + "\n"
    output += "CLASE: " + simbolo.clase + "\n"
    output += "NOCIÓN:\n"
    for noc in simbolo.nocion:
        output += "\t- " + noc + "\n"
    output += "IMPACTO:\n"
    for imp in simbolo.impacto:
        output += "\t- " + imp + "\n"
    if simbolo.especializaciones:
        output += "Especializaciones:\n"
        for esp in simbolo.especializaciones:
            output += esp.nombre + "\n"
    return output

def imprimir_lel():
    output = ""
    for sc in simbolos_confirmados:
        output += get_simbolo_string(sc)
    return output

output= imprimir_lel().replace("\x1b[4m","")
output=output.replace("\x1b[0m","")
with open('output.txt', 'w', encoding='utf-8') as f:
        f.write(output)
files.download('output.txt')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>